In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

In [2]:
# Import API key
from config import weather_api_key

In [3]:
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [4]:
# Output File (CSV)
output_data_file = "output_data/cities.csv"

In [5]:
# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Generate Cities List

In [6]:
 # List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
#pprint(cities)

631

# Perform API Calls

In [7]:
lat = []
lon = []
temp = []
wind_speed = []
max_temp = []
humidity = []
clouds = []
country = []
city_dt = []
name = []
id =[]

In [8]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
city = cities
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [ ]:
#Perform a weather check on each city using a series of successive API calls.
#Include a print log of each city as it's being processed (with the city number and city name).
# IT IS GETTING INTERUPTED AT CITY 6: 
# {'cod': '404' , 'message' : 'city not found'}
for each_city in city:
    response = requests.get(query_url + each_city).json()
    pprint(response)
    try:
        city_dt.append(response["dt"])
        id.append(response["id"])
        name.append(response["name"])
        temp.append(response['main']['temp'])
        lat.append(response["coord"]["lat"])
        lon.append(response['coord']['lon'])
        wind_speed.append(response["wind"]["speed"])
        max_temp.append(response["main"]["temp_max"])
        humidity.append(response["main"]["humidity"])
        clouds.append(response["clouds"]["all"])
        country.append(response["sys"]["country"])
        print("Found")
        #print(f"You got {name}")
    except KeyError: 
        print("Not found")
        pass
    

# Convert Raw Data to DF

In [ ]:
#Export the city data into a .csv.
#LAT, LONG, MAX TEMP, HUMIDITY, CLOUD COVERAGE, WIND SPEED, CITY COUNTRY, CITY DATETIME
#Display the DataFrame

# Inspect Data and Remove Cities Where Humnidity is > 100%

In [ ]:
#Skip this step if no cities are > 100% humidity 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
Int64Index([], dtype='int64')

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

# Plot The Data

In [ ]:
#Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
#Save the plotted figures as .pngs.

# Latitude vs. Temperature Plot

In [ ]:
# PLOT THEN ADD A FEW SENTENCES "EXPLAINING WHAT THE CODE IS ANALYZING"

# Latitude vs. Humidity Plot

In [ ]:
# PLOT THEN ADD A FEW SENTENCES "EXPLAINING WHAT THE CODE IS ANALYZING"

# Latitude vs. Cloudiness Plot

In [ ]:
# PLOT THEN ADD A FEW SENTENCES "EXPLAINING WHAT THE CODE IS ANALYZING"

# Latitude vs. Wind Speed Plot

In [ ]:
# PLOT THEN ADD A FEW SENTENCES "EXPLAINING WHAT THE CODE IS ANALYZING"

# Linear Regression

# Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# The r-value is: 0.46937863826630627

In [ ]:
#PUT PLOT HERE AND ADD A FEW SENTENCES TO DESCRIBE RELATIONSHIPS OR OTHER OBSERVATIONS

# Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# The r-value is: 0.6082708688906918

In [ ]:
#PUT PLOT HERE AND ADD A FEW SENTENCES TO DESCRIBE RELATIONSHIPS OR OTHER OBSERVATIONS

# Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#The r-value is: 0.000543987878808455

In [ ]:
#PUT PLOT HERE AND ADD A FEW SENTENCES TO DESCRIBE RELATIONSHIPS OR OTHER OBSERVATIONS

# Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# The r-value is: 0.014985174297345524

In [ ]:
#PUT PLOT HERE AND ADD A FEW SENTENCES TO DESCRIBE RELATIONSHIPS OR OTHER OBSERVATIONS

# Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# The r-value is: 0.0020898727664228514

In [ ]:
#PUT PLOT HERE AND ADD A FEW SENTENCES TO DESCRIBE RELATIONSHIPS OR OTHER OBSERVATIONS

# Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# The r-value is: 0.0035263772085582894

In [ ]:
#PUT PLOT HERE AND ADD A FEW SENTENCES TO DESCRIBE RELATIONSHIPS OR OTHER OBSERVATIONS

# Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# The r-value is: 0.04019723576144484

In [ ]:
#PUT PLOT HERE AND ADD A FEW SENTENCES TO DESCRIBE RELATIONSHIPS OR OTHER OBSERVATIONS

# Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#  The r-value is: 0.017128274759839232

In [ ]:
#PUT PLOT HERE AND ADD A FEW SENTENCES TO DESCRIBE RELATIONSHIPS OR OTHER OBSERVATIONS